Lateness Report [link](https://gttwiki.atlassian.net/wiki/spaces/GD/pages/2492956684/Lateness+report)

Approach 2 (compare time late for the last stop for TSPon vs TSPoff)

In [1]:
%load_ext lab_black
import numpy as np
import pandas as pd
import pyspark
import scipy.stats

spark = pyspark.sql.SparkSession.builder.appName(
    "LatnessHypothesisCountLate [pyspark]"
).getOrCreate()
spark

In [2]:
!tree -h --sort=size /var/lib/gtt/2020_11_8_to_2020_11_14

/var/lib/gtt/2020_11_8_to_2020_11_14
├── [4.0K]  CMS
│   ├── [2.0G]  opticomdevicelog.csv
│   ├── [607K]  intersectionstatusreport.csv
│   ├── [ 25K]  opticomdevicelog_data_dictonary.docx
│   └── [ 15K]  intersectionstatusreport_data_dictionary.docx
└── [4.0K]  CVP
    ├── [5.4G]  tripdatas.csv
    ├── [3.3M]  triplogs.csv
    ├── [1.4M]  devices.csv
    ├── [ 18K]  triplogs_data_dictionary.docx
    ├── [ 15K]  devices_data_dictionary.docx
    └── [ 15K]  tripdatas_data_dictionary.docx

2 directories, 10 files


In [3]:
%%time
spark.sql(
    """
    create table if not exists tripdatas using csv options (
        header = true,
        inferSchema = true,
        path = '/var/lib/gtt/2020_11_8_to_2020_11_14/CVP/tripdatas.csv'
    );
    """
)

CPU times: user 10.1 ms, sys: 3.54 ms, total: 13.6 ms
Wall time: 1min 6s


DataFrame[]

In [4]:
%%time
spark.sql(
    """
    create table if not exists triplogs using csv options (
        header = true,
        inferSchema = true,
        path = '/var/lib/gtt/2020_11_8_to_2020_11_14/CVP/triplogs.csv'
    );
    """
)

CPU times: user 1.07 ms, sys: 272 µs, total: 1.34 ms
Wall time: 426 ms


DataFrame[]

In [5]:
%%time
spark.sql(
    """
    cache table breadcrumbs as (
        select deviceid || '_' || logid as gd_uid,
               count(distinct time)     as count
        from tripdatas
        where event = 'GPS'
        group by 1
        order by 1
    );
    """
);

CPU times: user 12.6 ms, sys: 3.14 ms, total: 15.8 ms
Wall time: 1min 23s


DataFrame[]

In [6]:
%%time
spark.sql(
    """
    cache table triplogs_good as (
        with --
             triplogs as (
                 select countlate,
                        countstopshit,
                        direction,
                        duration,
                        endstatus,
                        routename,
                        stops,
                        tspmode,
                        valid,
                        to_timestamp(starttime)                    as gd_starttime,
                        to_timestamp(endtime)                      as gd_endtime,
                        date_part('hour', to_timestamp(starttime)) as gd_hour,
                        deviceid || '_' || logid                   as gd_uid
                 from triplogs
             ),
             triplogs_good as (
                 select distinct triplogs.*
                 from triplogs
                          left join breadcrumbs using (gd_uid)
                 where valid
                   and gd_starttime < gd_endtime
                   and endstatus = 'completed'
                   and tspmode != 'normal'
                   and duration > 0
                   and countstopshit / stops > 0.7
                   and breadcrumbs.count / duration > 0.7
             ),
             three_sigma as (
                 select mean(duration) - 3 * stddev(duration) as min,
                        mean(duration) + 3 * stddev(duration) as max
                 from triplogs_good
             )
        select triplogs_good.*
        from triplogs_good,
             three_sigma
        where duration > three_sigma.min
          and duration < three_sigma.max
    );
    """
);

CPU times: user 0 ns, sys: 2.88 ms, total: 2.88 ms
Wall time: 4.74 s


DataFrame[]

## Whole trip

In [7]:
%%time
spark.sql(
    """
    cache table whole_trip as (
        select routename,
               direction,
               stops,
               collect_list(case when tspmode = 'alwaysOn' then countlate end)  as countlate_alwayson,
               collect_list(case when tspmode = 'alwaysOff' then countlate end) as countlate_alwaysoff
        from triplogs_good
        where tspmode in ('alwaysOn', 'alwaysOff')
        group by 1, 2, 3
        having size(countlate_alwayson) > 3
           and size(countlate_alwaysoff) > 3
        order by 1, 2, 3
    );
    """
);

CPU times: user 0 ns, sys: 3.05 ms, total: 3.05 ms
Wall time: 5.58 s


DataFrame[]

In [8]:
def statistical_testing(row):
    try:
        row["mannwhitneyu_p"] = scipy.stats.mannwhitneyu(
            row["countlate_alwayson"], row["countlate_alwaysoff"], alternative="less"
        ).pvalue
    except ValueError:
        row["mannwhitneyu_p"] = np.nan

    try:
        row["ttest_p"] = scipy.stats.ttest_ind(
            row["countlate_alwayson"], row["countlate_alwaysoff"], alternative="less"
        ).pvalue
        if (
            np.abs(
                np.sqrt(np.var(row["countlate_alwayson"], ddof=1))
                - np.sqrt(np.var(row["countlate_alwaysoff"], ddof=1))
            )
            > 3
        ):
            row["ttest_p"] = np.nan
    except ValueError:
        row["ttest_p"] = np.nan

    row["countlate_alwayson"] = str(row["countlate_alwayson"])[:25]
    row["countlate_alwaysoff"] = str(row["countlate_alwaysoff"])[:25]
    return row

In [9]:
whole_trip_results = (
    spark.table("whole_trip")
    .toPandas()
    .set_index(["routename", "direction", "stops"])
    .apply(statistical_testing, axis=1)
)
display(
    whole_trip_results.filter(regex=r"_p$")
    .apply(
        lambda s: pd.Series(
            {
                "results": f"{(s < 0.05).sum() / s.notna().sum():.2%} routes are significantly faster",
                "errors": f"{s.isna().sum() / len(s):.2%} routes errored out",
            }
        )
    )
    .T
)
with pd.option_context("display.max_rows", None):
    display(
        whole_trip_results.style.applymap(
            lambda x: (
                "background-color:rgba(256,0,0,.2)"
                if x > 0.05
                else "background-color:rgba(0,256,0,.2)"
                if x < 0.05
                else ""
            ),
            subset=["mannwhitneyu_p", "ttest_p"],
        )
    )

,results,errors
mannwhitneyu_p,15.87% routes are significantly faster,7.35% routes errored out
ttest_p,2.22% routes are significantly faster,33.82% routes errored out


## With hourly separation

In [10]:
%%time
spark.sql(
    """
    cache table hourly_separation as (
        select routename,
               direction,
               stops,
               gd_hour,
               collect_list(case when tspmode = 'alwaysOn' then countlate end)  as countlate_alwayson,
               collect_list(case when tspmode = 'alwaysOff' then countlate end) as countlate_alwaysoff
        from triplogs_good
        where tspmode in ('alwaysOn', 'alwaysOff')
        group by 1, 2, 3, 4
        having size(countlate_alwayson) > 3
           and size(countlate_alwaysoff) > 3
        order by 1, 2, 3, 4
    );
    """
);

CPU times: user 2.12 ms, sys: 434 µs, total: 2.56 ms
Wall time: 6.82 s


DataFrame[]

In [11]:
hourly_separation_results = (
    spark.table("hourly_separation")
    .toPandas()
    .set_index(["routename", "direction", "stops", "gd_hour"])
    .apply(statistical_testing, axis=1)
)
display(
    hourly_separation_results.filter(regex=r"_p$")
    .apply(
        lambda s: pd.Series(
            {
                "results": f"{(s < 0.05).sum() / s.notna().sum():.2%} routes are significantly faster",
                "errors": f"{s.isna().sum() / len(s):.2%} routes errored out",
            }
        )
    )
    .T
)
with pd.option_context("display.max_rows", None):
    display(
        hourly_separation_results.style.applymap(
            lambda x: (
                "background-color:rgba(256,0,0,.2)"
                if x > 0.05
                else "background-color:rgba(0,256,0,.2)"
                if x < 0.05
                else ""
            ),
            subset=["mannwhitneyu_p", "ttest_p"],
        )
    )

,results,errors
mannwhitneyu_p,3.86% routes are significantly faster,27.64% routes errored out
ttest_p,4.38% routes are significantly faster,57.45% routes errored out
